라이브러리 설치  
pip install transformers torch Pillow scipy  
테스트용  
  
TODO  
 label DB에서 추출해서 사용 < 일단은 했음  
 이미지 받는거 변수로 해야함  
 GPU사용 확인  
 찾은 사물 현재 파이썬 set으로 사용 파일로 넘길지 이 노트북에서 끝까지 처리할지 정해야함. < 일단 이 파일에 했음

pip install pymysql  
CREATE DATABASE kculture;

In [4]:
input_year = 1930
image_path = "raw image/1930.jpg"

In [1]:
import pymysql

conn = pymysql.connect(host='127.0.0.1', user='root', password='0000', db='kculture', charset='utf8') # DB 연결, password 본인 DB 비밀번호로 변경
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS ObjectList (
    name VARCHAR(255) NOT NULL PRIMARY KEY,
    start_year INT NOT NULL,
    end_year INT NOT NULL DEFAULT (YEAR(CURDATE())),
    description TEXT NULL
)""")

cur.execute("""INSERT INTO ObjectList (name, start_year, description) VALUES
    ('band', 1970, 'wound-protecting medical supplies'),
    ('drain', 1960, 'a grid-shaped iron drain'),
    ('Milks', 1989, 'korean drink'),
    ('padded jacket', 1992, 'nylon padding'),
    ('boom microphone', 1929, 'a microphone attached to a long pole'),
    ('tablet computer', 2010, 'iPad or Galaxy Tab etc.'),
    ('buldak', 2010, 'Spicy korean ramen'),
    ('seoul milk', 1974, 'milk sold in square pack'),
    ('band-aid', 1920, 'A small bandage used to cover a cut or a scratch.'),
    ('mouse', 1964, 'A hand-held pointing device for computers.'),
    ('mouse pad', 1970, 'A surface for placing and moving a computer mouse.'),
    ('USB Flash Drive', 2000, 'A small storage device.'),
    ('smart watch', 2012, 'A wristwatch with computing capabilities.'),
    ('sunglasses', 1929, 'Eyewear designed to protect the eyes from the sun.'),
    ('credit card', 1958, 'A plastic card used to make purchases on credit.'),
    ('bluetooth speaker', 2001, 'A portable speaker that connects wirelessly.'),
    ('air conditioner', 1966, 'A device that cools and dehumidifies the air.')
    ON DUPLICATE KEY UPDATE start_year=VALUES(start_year), description=VALUES(description)
    """)

conn.commit()

cur.execute("SELECT * from ObjectList")

rows = cur.fetchall()
for row in rows:
    print(row)
    
conn.close()

('air conditioner', 1966, 2025, 'A device that cools and dehumidifies the air.')
('band', 1970, 2025, 'wound-protecting medical supplies')
('band-aid', 1920, 2025, 'A small bandage used to cover a cut or a scratch.')
('bluetooth speaker', 2001, 2025, 'A portable speaker that connects wirelessly.')
('boom microphone', 1929, 2025, 'a microphone attached to a long pole')
('buldak', 2010, 2025, 'Spicy korean ramen')
('credit card', 1958, 2025, 'A plastic card used to make purchases on credit.')
('drain', 1960, 2025, 'a grid-shaped iron drain')
('Milks', 1989, 2025, 'korean drink')
('mouse', 1964, 2025, 'A hand-held pointing device for computers.')
('mouse pad', 1970, 2025, 'A surface for placing and moving a computer mouse.')
('padded jacket', 1992, 2025, 'nylon padding')
('seoul milk', 1974, 2025, 'milk sold in square pack')
('smart watch', 2012, 2025, 'A wristwatch with computing capabilities.')
('sunglasses', 1929, 2025, 'Eyewear designed to protect the eyes from the sun.')
('tablet com

In [5]:
from PIL import Image
import torch
from transformers import pipeline

if torch.cuda.is_available():
    print(f"GPU ({torch.cuda.get_device_name(0)})")
    device_option = 0
else:
    print("CPU")
    device_option = -1

# 'zero-shot-object-detection' 파이프라인 불러오기
#    - model: 사용할 모델을 지정합니다. 'google/owlv2-base-patch16-ensemble'은 성능이 좋은 최신 모델입니다.
#    - task: 수행할 작업을 지정합니다.
detector = pipeline(model="google/owlv2-base-patch16-ensemble", task="zero-shot-object-detection", device=device_option)
print("모델 로딩 완료.")

# DB에서 객체 목록 불러오기
conn = pymysql.connect(host='127.0.0.1', user='root', password='0000', db='kculture', charset='utf8') # DB 연결, password 본인 DB 비밀번호로 변경
cur = conn.cursor()
cur.execute("SELECT name from ObjectList")
candidate_labels = []
rows = cur.fetchall()
for row in rows:
    candidate_labels.append(row[0])
conn.close()

try:
    image = Image.open(image_path)
    print(f"'{image_path}' 이미지를 성공적으로 불러왔습니다.")

    # 이미지에서 찾고 싶은 객체 목록 정의
    #    - 이 목록을 수정하여 원하는 객체를 찾을 수 있습니다.
    #    - "a photo of a cat" 처럼 구체적으로 작성하면 정확도가 올라갑니다.
    print(f"탐색할 객체: {candidate_labels}")

    # 모델 실행하여 객체 탐지
    #     - 이미지와 객체 후보 목록을 모델에 전달합니다.
    print("객체 탐지를 시작합니다...")
    predictions = detector(image, candidate_labels=candidate_labels)
    print("객체 탐지 완료.")

    # 결과 출력
    #    - 모델이 찾은 각 객체의 정보를 보기 쉽게 출력합니다.
    print("\n--- 탐지 결과 ---")
    if not predictions:
        print("이미지에서 지정된 객체를 찾지 못했습니다.")
    else:
        labels = []
        for prediction in predictions:
            labels.append(prediction["label"])
            
            # 신뢰도 점수가 0.1 이상인 결과만 출력 (불필요한 결과 제거)
            if prediction["score"] > 0.1:
                labels.append(prediction["label"])
        set_labels = set(labels)  # 중복 제거
        print(set_labels)

except FileNotFoundError:
    print(f"오류: '{image_path}' 파일을 찾을 수 없습니다. 코드와 같은 폴더에 이미지 파일이 있는지 확인하세요.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

CPU


Device set to use cpu


모델 로딩 완료.
'raw image/1930.jpg' 이미지를 성공적으로 불러왔습니다.
탐색할 객체: ['air conditioner', 'band', 'band-aid', 'bluetooth speaker', 'boom microphone', 'buldak', 'credit card', 'drain', 'Milks', 'mouse', 'mouse pad', 'padded jacket', 'seoul milk', 'smart watch', 'sunglasses', 'tablet computer', 'USB Flash Drive']
객체 탐지를 시작합니다...
객체 탐지 완료.

--- 탐지 결과 ---
{'air conditioner'}


In [6]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='0000', db='kculture', charset='utf8') # DB 연결, password 본인 DB 비밀번호로 변경
cur = conn.cursor()

cur.execute("SELECT * from ObjectList")

format_strings = ','.join(['%s'] * len(set_labels))
sql = "SELECT name, start_year, description FROM ObjectList WHERE name IN (%s)" % format_strings
cur.execute(sql, tuple(set_labels) )

rows = cur.fetchall()
for row in rows:
    if row[1] > input_year:
        print("이름 : {}, 출시 연도: {}, 설명: {}\n".format(row[0], row[1], row[2]))

conn.close()

이름 : air conditioner, 출시 연도: 1966, 설명: A device that cools and dehumidifies the air.

